# GOLD - DASHBOARD LEVEL TABLES
# Purpose: CREATE 3 TABLES TO ANSWER BUSINESS QUESTION
# Source: DELTA TABLE
# Output: DELTA TABLES
## - gold.houses_per_quartile
## - gold.properties in each category
## - gold.changes in cateogry

## CONFIG/PARAMETERS

In [0]:
%sql
USE CATALOG harris_county_catalog

## TABLE CREATION

This table shows the amount of properties in each quartile

In [0]:
%sql
CREATE TABLE gold.properties_in_quartile
SELECT 
  quartile
  , COUNT(*) as amount_of_properties_in_quartile
FROM harris_county_catalog.gold.cummulative_scd
WHERE end_date = '2025-01-01'
GROUP BY quartile

num_affected_rows,num_inserted_rows


This table categorizes each house depending on how many years they have remained in the same quartile, then counts them

Categories:
- Very very stable (8 years in the same quartile)
- Very stable (4 years in the same quartile)
- Stable (2 years in the same quartile)
- Variable (less than 2 years in the same quartile)

In [0]:
%sql
CREATE TABLE gold.stability_categorization
with interval_value(
  SELECT
    *
    , CAST(end_date - start_date AS INT ) AS `interval`
  FROM harris_county_catalog.gold.cummulative_scd
)
, price_categorization_table
(
  SELECT 
    *
    , CASE 
        WHEN `interval` > 2920
          THEN 'Very very stable price'
        WHEN `interval` > 1460
          THEN 'Very stable price'
        WHEN `interval` > 730
          THEN 'stable price'
        ELSE 'variable price'
    END as price_categorization
  FROM interval_value
  WHERE end_date = '2025-01-01'
  AND quartile = '25-50%'
-- AND `interval` > 2500
)
, grouped_values
  (
    SELECT 
    price_categorization
    , COUNT(price_categorization) AS amount_of_prop_in_category
    , SUM(`interval`) AS total_days_per_category
  FROM price_categorization_table
  GROUP BY price_categorization
  ORDER BY amount_of_prop_in_category
)
SELECT 
  price_categorization
  , amount_of_prop_in_category
  -- , total_days_per_category/(amount_of_prop_in_category*365) as average_years_in_category
FROM grouped_values

num_affected_rows,num_inserted_rows


This table counts how many times a property has changed quartiles. Then counts how many properties have changed the same amount of times

In [0]:
%sql
CREATE TABLE gold.amount_of_quartile_changes
with changes_in_quartiles(
  SELECT 
    *
    , row_number() OVER (PARTITION BY dim_account_number ORDER BY start_date) AS changes_in_quartiles
  FROM harris_county_catalog.gold.cummulative_scd
  WHERE quartile = '25-50%'
  )
, metrics_t
  (
  SELECT 
    MAX(changes_in_quartiles) as maximum 
  FROM changes_in_quartiles
  GROUP BY dim_account_number)
, maximums_counted 
  (
  SELECT 
    maximum
    , COUNT(*) counts
  FROM metrics_t
  GROUP BY maximum
  )
SELECT
  maximum - 1 as category_changes 
  , counts
FROM maximums_counted
ORDER BY maximum

num_affected_rows,num_inserted_rows
